In [1]:
import pandas as pd
import folium
import json
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from folium.plugins import HeatMap

/opt/app-root/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
file_path = 'data/'
kids = pd.read_csv(file_path+'오산시_어린이집_유치원현황.csv')
school = pd.read_csv(file_path+'오산시_학교위치정보.csv')
child =  pd.read_csv(file_path+'오산시_어린이보호구역.csv')

In [18]:
school = school[school['학교구분']=='초등학교']

In [8]:
child.head()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749


어린이 보호구역 heatmap

In [13]:
data =[]
for i in child.index:
    data.append([child.보호구역_위도[i],child.보호구역_경도[i]])

In [24]:
m = folium.Map(
    location = [37.149799,127.077116],
    zoom_start = 11,
)
geo_data= file_path+'오산시_법정경계(읍면동).geojson'
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#7CFC00',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
HeatMap(data).add_to(m)
m

In [19]:
school.head()

,학교구분,학교명,학교위치_경도,학교위치_위도
0,초등학교,원동초등학교,127.069699,37.135168
1,초등학교,운산초등학교,127.077414,37.152500
2,초등학교,화성초등학교,127.068858,37.164615
12,초등학교,가수초등학교,127.047749,37.147919
13,초등학교,다온초등학교,127.090752,37.151698


In [17]:
kids.head()

,시설구분,시설타입,시설명,시설위치_경도,시설위치_위도
0,어린이집,민간,해오름어린이집,127.084336,37.132383
1,어린이집,가정,햇살가득어린이집,127.041817,37.182164
2,어린이집,가정,행복한아이들어린이집,127.079494,37.152891
3,어린이집,가정,휴튼사과나무어린이집,127.049638,37.177976
4,어린이집,직장,(주)아모레퍼시픽 오산어린이집,127.040252,37.165311


In [21]:
# 어린이집과 유치원
kids.시설구분.value_counts()

어린이집    257
유치원      50
Name: 시설구분, dtype: int64

In [20]:
# 어린이집 유치원 타입갯수
kids.시설타입.value_counts()

가정        150
민간         59
국공립        40
공립         25
사립         25
직장          5
법인·단체       2
사회복지법인      1
Name: 시설타입, dtype: int64

어린이집과 유치원 지도 시각화

In [37]:
geo_data= file_path+'오산시_법정경계(읍면동).geojson'
center = [37.149799,127.077116]

m = folium.Map(location=center, zoom_start=11)
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#7CFC00',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
    

for i in kids.index:
    if kids.loc[i,'시설구분'] == '어린이집':
        color = 'lightred'
        icon = 'child'
    else:    
        color = 'lightblue'
        icon = 'child'
    folium.Marker(
        location = kids.loc[i, ['시설위치_위도', '시설위치_경도']],
        radius = 200,
        icon=folium.Icon(color = color,icon = icon, prefix='fa'),
        tooltip = kids.loc[i,'시설명']
    ).add_to(m)


m

In [30]:
# 어린이집 + 유치원 + 초등학교 합치기
child_total = kids.drop('시설타입',axis=1)
child_total.columns=['구분','이름','경도','위도']

In [31]:
temp = school
temp.columns = ['구분','이름','경도','위도']

In [34]:
child_total = pd.concat([child_total,temp])

어린이집 + 유치원 + 초등학교 + 어린이 보호구역

In [35]:
child_total.구분.value_counts()

어린이집    257
유치원      50
초등학교     24
Name: 구분, dtype: int64

In [50]:
child_total = child_total.reset_index()

In [56]:
child_total= child_total.drop('index',axis=1)

In [59]:
geo_data= file_path+'오산시_법정경계(읍면동).geojson'
center = [37.149799,127.077116]

m = folium.Map(location=center, zoom_start=11)
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#7CFC00',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases = ['동'])
).add_to(m)
    

for i in child_total.index:
    if child_total.loc[i,'구분'] == '어린이집':
        color = 'lightred'
        icon = 'child'
    elif child_total.loc[i,'구분'] == '유치원':
        color = 'cadetblue'
        icon = 'child'
    else:    
        color = 'lightblue'
        icon = 'child'
    folium.Marker(
        location = child_total.loc[i, ['위도', '경도']],
        radius = 200,
        icon=folium.Icon(color = color,icon = icon, prefix='fa'),
        tooltip = child_total.loc[i,'이름']
    ).add_to(m)
HeatMap(data).add_to(m)
m

어린이집이 가정인경우는 보호구역이 아니다. 그래서 어떤 유형의 어린이집 + 유치원일경우 보호구역 안에있는가?

In [89]:
geo_data= file_path+'오산시_법정경계(읍면동).geojson'
center = [37.149799,127.077116]

m = folium.Map(location=center, zoom_start=11)
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#7CFC00',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases = ['동'])
).add_to(m)
    

for i in kids.index:
    if kids.loc[i,'시설타입'] == '가정':
        color = 'lightred'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '민간':
        color = 'cadetblue'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '국공립':
        color = 'green'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '공립':
        color = 'pink'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '사립':
        color = 'lightgray'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '직장':
        color = 'purple'
        icon = 'child'
    elif kids.loc[i,'시설타입'] == '법인·단체':
        color = 'black'
        icon = 'child'
    else:    
        color = 'lightblue'
        icon = 'child'
    folium.Marker(
        location = kids.loc[i, ['시설위치_위도', '시설위치_경도']],
        radius = 200,
        icon=folium.Icon(color = color,icon = icon, prefix='fa'),
        tooltip = kids.loc[i,'시설명'] + ':' + kids.loc[i,'시설타입']
    ).add_to(m)
    

for i in child.index:
    folium.Circle(
        location = child.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 50,
        color='red',
        tooltip = child.loc[i,'시설명'] + ':' + child.loc[i,'시설종류']
    ).add_to(m)
m

어린이집일경우 : 국공립, 유치원은 민간,국공립 상관없음 => 어린이 보호구역 <br>
어린이집이 민간, 가정일경우 => 어린이 보호구역 해당되지 않음

초등학교 통학구에 초등학교 + 어린이 보호구역 위치

In [115]:
geo_data= file_path+'오산시_초등학교_통학구.geojson'
center = [37.149799,127.077116]

m = folium.Map(location=center, zoom_start=11)
geo_data2= file_path+'오산시_법정경계(읍면동).geojson'
folium.GeoJson(
    geo_data2,
    style_function=lambda feature: {
        'fillColor': '#FF99FF',
        'color': '#CC33CC',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases = ['동'])
).add_to(m)

folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': '#ADFF2F',
        'color': '#009900',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(fields=['학구명'])
).add_to(m)

# 보호구역 도로폭이 넓을수록 원크기도 커짐
for i in child.index:
    folium.Circle(
        location = child.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = child.loc[i,'보호구역도로폭'].item()*10,
        color='red',
        tooltip = child.loc[i,'시설명']
    ).add_to(m)

for i in school.index:
    folium.Marker(
        location = school.loc[i, ['위도', '경도']],
        tooltip = school.loc[i,'이름'],
        icon=folium.Icon(color = 'darkgreen',icon = 'child',prefix='fa'),
    ).add_to(m)
m

In [113]:
type(child.loc[1,'보호구역도로폭'].item())

int